In [31]:
import json
from coinbase.rest import RESTClient
from json import dumps
import pandas as pd

In [51]:
# Get demo API key
def get_demo_key(key_type):
    f = open(f"/home/vikas/Documents/Demo_{key_type}_CG_key.json")
    key_dict = json.load(f)
    return key_dict["name"], key_dict["privateKey"]

In [55]:
api_key, api_secret = get_demo_key("view")

view_client = RESTClient(api_key = api_key,
                         api_secret = api_secret)

## Test API

In [53]:
server_time = view_client.get_unix_time()
print(dumps(server_time, indent = 4))

{
    "iso": "2024-09-02T19:36:08Z",
    "epochSeconds": "1725305768",
    "epochMillis": "1725305768873"
}


In [54]:
view_client.get_api_key_permissions()

{'can_view': True,
 'can_trade': False,
 'can_transfer': False,
 'portfolio_uuid': 'd083ea15-3f44-54da-8e9e-2a5a9bdccb32'}

## Get trades for a given pair

In [58]:
# btc_usd = view_client.get_public_market_trades("BTC-USD", 2)
# print(dumps(btc_usd, indent = 4))

In [56]:
def get_latest_trades(pair, num_trades):
    
    all_trades = view_client.get_public_market_trades(pair, num_trades)
    
    return pd.DataFrame(all_trades["trades"])   

In [57]:
df_btc_usd = get_latest_trades("ETH-EUR", 5)
df_btc_usd

,trade_id,product_id,price,size,time,side,bid,ask
0,53256104,ETH-EUR,2283.22,0.04172349,2024-09-02T19:36:45.227154Z,SELL,,
1,53256103,ETH-EUR,2282.98,0.07098991,2024-09-02T19:36:44.097968Z,SELL,,
2,53256102,ETH-EUR,2282.91,0.14592355,2024-09-02T19:36:44.097968Z,SELL,,
3,53256101,ETH-EUR,2282.29,0.02083365,2024-09-02T19:36:26.728480Z,SELL,,
4,53256100,ETH-EUR,2281.54,0.0714551,2024-09-02T19:36:22.666133Z,BUY,,


## Get accounts

In [46]:
#client.get_accounts()

## Get portfolios

In [50]:
client.get_portfolios()

{'portfolios': [{'name': 'Default',
   'uuid': 'd083ea15-3f44-54da-8e9e-2a5a9bdccb32',
   'type': 'DEFAULT',
   'deleted': False},
  {'name': 'Demo portfolio',
   'uuid': '581b97fc-d5a9-4b69-bb3c-1c9853fb9b71',
   'type': 'CONSUMER',
   'deleted': False}]}

## Check portfolio breakdown

In [60]:
api_key, api_secret = get_demo_key("trade")

trade_client = RESTClient(api_key = api_key,
                          api_secret = api_secret)

In [ ]:
trade_client.get_portfolio_breakdown(